# MAE 271: Lab 2 - Inverted pendulum with vertical base excitation
### Cooper Cook & Joshua Booth

## Problem Introduction
The simulation problem below presents a vertical inverted pendulum with an excited base. This problem becomes interesting due to the fact that given certain initial angles from vertical and the right amplitudes and frequency's of excitation, the pendulum stays upright without falling over. Simulations provided below model this behavior and display the proper initial conditions that promote stabilization of the inverted mass. Animations are created to display the inverted pendulum stabilizing across variations in initial conditions.

## Simulation Setup
Below can be seen two images that setup the problem we are looking at. The diagram of the model can be seen with the excited base moving up the Y axis and inverted mass connected with a rod free to move in the X-Y plane. The mass is only influenced by the force of gravity and restricted by the connection to the exicted base. The angle $\theta$ is defined as positive from the vertical Y-axis moving clockwise in the diagram.

![diagram](image.png)

The bond graph below shows the physical model of the system and how each part is interacting such that we can derive equations to solve for the motion of the inverted mass.

![bond graph](image-1.png)

### Equations used for simulations

Velocity equations for X, Y, and $\theta$ directions

$$v_y = \dot{y} - l \dot{\theta} \sin(\theta) $$
$$v_x = l \dot{\theta} \cos(\theta) $$
$$ \dot{\theta} = \frac{p_x}{l\cos(\theta)\ m} $$

Momentum equations

$$p_x = m l \dot{\theta} \cos(\theta) $$
$$p_y = m\left[\dot{y} - \frac{\sin(\theta)\ p_x}{\cos(\theta)\, m}\right] $$

Derivatives of momentum equations
$$\dot{p_y} = m\left[\ddot{y} - \frac{\sin(\theta)\ \dot{p_x}}{\cos(\theta)\ m} - \frac{p_x\ \dot{\theta}}{m\ \cos^2{\theta}} \right] $$
$$\dot{p_x} = m g \sin(\theta)\cos(\theta) + m \ddot{y}\sin(\theta)\cos(\theta) - \frac{\sin(\theta)}{\cos(\theta)} \dot{\theta} p_x$$




## Code and figures

### Global Parameters
Define global parameters and initial conditions for the system:

In [ ]:
import numpy as np

# -----------------------------
# Global Parameters
# -----------------------------
params: dict[str, float] = {}
params["l_arm"] = 1.0   # length of pendulum arm [m]
params["m_mass"] = 1.0  # mass of pendulum mass [kg]
params["g"] = 9.8       # gravitational force [m/s^2]

params["theta"] = 0     # initial pendulum angle [deg] measured clockwise from vertical
params["amplitude"] = 0 # oscillator amplitude [m]
params["frequency"] = 0 # oscillator frequency [Hz]

# Time
t_start: float = 0
t_end: float = 2
t_increment: float = 0.010

t_span = (t_start, t_end)
t_eval = np.arange(min(t_span), max(t_span)+t_increment, t_increment)

### Define Derivative Function
To solve this non-linear system of ordinary differential equations, we need to start from an initial state, numerically integrate along the state derivatives to find the next state, and repeat as desired

For integration, we will use `scipy.integrate`'s `solve_ivp()`, which takes in a range of time (`t_span`) through which to integrate, a function handle (`func`) that takes in the time and state and returns the derivatives, and the initial condition (`initial`) of the state.

Since we wish to solve this problem using several different parameters, we will define a function `get_func()` that returns a function handle that we can pass to `solve_ivp()`. We also return a function that includes additional state information.

In [ ]:
from copy import deepcopy
import numpy as np

def get_func(params: dict[str, float]):
    # Make a copy of the input parameters to ensure the returned function 
    # does not change when the original parameters dictionary changes
    params = deepcopy(params)

    # Extract parameters from dict
    theta = params["theta"]
    ampl = params["amplitude"]
    freq = params["frequency"]*2*np.pi # Convert from Hz to rad/s
    leng = params["l_arm"]
    mass = params["m_mass"]
    g = params["g"]

    # Get initial condition (add position into the system so solve_ivp can integrate it for us)
    px = 0
    initial = [px, theta]

    # Create derivative calculator function to be used by solve_ivp()
    # state = [px, theta]
    def func(t, state):
        state = deepcopy(state)
        px = state[0]
        theta = state[1]

        # Sine and cosine of theta, for computational speed
        sin_theta = np.sin(theta)
        cos_theta = np.cos(theta)
        tan_theta = np.tan(theta)

        # Position of cart and its derivatives
        X = 0
        Y = ampl*np.sin(freq*t)
        d_Y = ampl*freq*np.cos(freq*t)
        dd_Y = -(freq**2)*Y

        # State derivatives
        d_theta = px/(leng*cos_theta*mass)
        d_px = mass*g*sin_theta*cos_theta + mass*dd_Y*sin_theta*cos_theta - tan_theta*d_theta*px

        # Other state variables
        x = X + leng*np.sin(theta) # x-position of pendulum mass
        y = Y + leng*np.cos(theta) # y-position of pendulum mass
        py = mass*(d_Y - tan_theta*px/mass) # y-momentum of pendulum mass
        d_py = mass*(dd_Y - tan_theta*d_px/mass - px*d_theta/(mass*cos_theta**2)) # first time-derivative of y-momentum of pendulum mass
        
        # Concatenate state derivatives
        d_state = [d_px, d_theta]

        # Create current state dict
        state = {}
        state["t"] = t
        state["X"] = X
        state["Y"] = Y
        state["x"] = x
        state["y"] = y
        state["px"] = px
        state["py"] = py
        state["theta"] = theta
        state["d_Y"] = d_Y
        state["d_px"] = d_px
        state["d_py"] = d_py
        state["d_theta"] = d_theta
        state["dd_Y"] = dd_Y
        state["tan_theta"] = tan_theta

        return d_state, state

    def func_wrap(t, state):
        '''
        Since solve_ivp() needs a function that only returns the state
        derivatives, we create a wrapper for func() that discards the rest of
        the state and returns only the state derivatives
        '''
        d_state, state = func(t, state)
        return d_state

    return func, func_wrap, initial
    

### Propagate Solution Using `solve_ivp()`
Now that we have a function that calculates the state derivatives given the state, we need to propagate the solution throughout time. The function `solve_problem()` was designed to use the `get_func()` function we defined earlier, calculate the solution using `scipy.integrate`'s `solve_ivp()`, and consolidate the interesting parts of the solution and the parameters given to define it. 

In [ ]:
from scipy.integrate import solve_ivp
import pandas as pd
import numpy as np


def solve_problem(
        params: dict[str, float],
        t_eval: np.ndarray,
        name: str,
        rtol: float = 1e-12,
        atol: float = 1e-12,
    ):

    func, func_wrap, initial = get_func(params)
    output = solve_ivp(
        func_wrap,
        (min(t_eval), max(t_eval)),
        initial,
        t_eval=t_eval,
        method="RK45",
        rtol=rtol,
        atol=atol,
    )

    ts = output.t
    ys = output.y

    states: list[dict[str, float]] = []
    for t, y in zip(ts, ys.T):
        d_state, state = func(t, y)
        states.append(state)

    df = pd.DataFrame(states)

    solution = {
        "params": params.copy(),
        "data": df,
        "name": name,
    }

    return solution


### Define Plotting Functions
To visualize the solutions, we can create a static plot or an animation of the pendulum. Since we have several solutions we would like to plot together, we create functions to consolidate the code. `plot_time()` and `plot_ani()` each take in a list of dictionaries outputted as the solution from `solve_problem()`.

In [ ]:
from typing import Any
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import pandas as pd


def plot_time(solutions):
    fig = plt.figure()
    ax = fig.add_subplot()
    for solution in solutions:
        name = solution["name"]
        df = solution["data"]

        t_vals = df.get("t").to_numpy()
        X_vals = df.get("X").to_numpy()
        Y_vals = df.get("Y").to_numpy()
        x_vals = df.get("x").to_numpy()
        y_vals = df.get("y").to_numpy()
        px_vals = df.get("px").to_numpy()
        py_vals = df.get("py").to_numpy()
        d_px_vals = df.get("d_px").to_numpy()
        d_py_vals = df.get("d_py").to_numpy()
        theta_vals = df.get("theta").to_numpy() * 180 / np.pi
        tan_theta_vals = df.get("tan_theta").to_numpy()

        # ax.plot(t_vals, x_vals, label=f"{name}: x")
        # ax.plot(t_vals, y_vals, label=f"{name}: y")
        # ax.plot(t_vals, d_px_vals, label=f"{name}: d_px")
        ax.plot(t_vals, theta_vals, label=f"theta: {name}")
        # theta_ax.plot(ts, tan_theta_vals, label="tan_theta")
    plt.legend()
    return fig


def plot_ani(solutions: list[dict[str, Any]], interval: int = 10):
    fig = plt.figure()
    ax = fig.add_subplot()
    all_x = np.array([])
    all_y = np.array([])

    # Initialize arrays to store the animation plots and data to use to update them
    num_solutions = len(solutions)
    plots: list[dict[str, Line2D]] = [{} for n in range(num_solutions)]
    data: list[dict[str, np.ndarray]] = [{} for n in range(num_solutions)]

    for i, solution in enumerate(solutions):
        name: str = solution["name"]
        df: pd.DataFrame = solution["data"]

        data[i]["t_vals"] = df.get("t").to_numpy()
        data[i]["X_vals"] = df.get("X").to_numpy()
        data[i]["Y_vals"] = df.get("Y").to_numpy()
        data[i]["x_vals"] = df.get("x").to_numpy()
        data[i]["y_vals"] = df.get("y").to_numpy()

        (plots[i]["cart"],) = ax.plot([], [], "o", c="b", label=f"cart: {name}")
        (plots[i]["line"],) = ax.plot([], [], "-", c="k")
        (plots[i]["pend"],) = ax.plot([], [], "o", label=f"pendulum: {name}")

        num_frames = len(data[i]["t_vals"])

        all_x = np.concatenate((all_x, data[i]["X_vals"].copy(), data[i]["x_vals"].copy()))
        all_y = np.concatenate((all_y, data[i]["Y_vals"].copy(), data[i]["y_vals"].copy()))

    def update_points(n):
        returns = []
        for i in range(num_solutions):
            plots[i]["cart"].set_data(([data[i]["X_vals"][n]], [data[i]["Y_vals"][n]]))

            plots[i]["line"].set_data(
                (
                    [data[i]["X_vals"][n], data[i]["x_vals"][n]],
                    [data[i]["Y_vals"][n], data[i]["y_vals"][n]],
                )
            )

            plots[i]["pend"].set_data(([data[i]["x_vals"][n]], [data[i]["y_vals"][n]]))
            returns.extend([plots[i]["cart"], plots[i]["line"], plots[i]["pend"]])

        returns = tuple(returns)
        return (*returns,)

    ani = animation.FuncAnimation(
        fig, update_points, num_frames, interval=interval, blit=True, repeat=True
    )

    min_x, max_x = min(all_x), max(all_x)
    min_y, max_y = min(all_y), max(all_y)
    plt.xlim(min_x - 0.5, max_x + 0.5)
    plt.ylim(min_y - 0.5, max_y + 0.5)
    plt.legend()

    return ani


## Results and Discussion
The results from the simulations above describe the behavior that we would be expecting from the model created. We can see that for set initial angles of $\theta$  = INSERT we were able to recover the inverted pendulum to stay upright. Starting at 90 was too far from vertical that no amplitude or excitation was able to recover it. Varying numbers of amplitude and frequency of the excitation.

## Conclusions
The inverted pendulum simulation is a great example of behavior that seems non-physical but can be a reality given the right set-up conditions to make it possible. It is also a great test case for bond graph modelling that enables simpler descriptions of the system to be simulated. As can be seen, the proper modelling and simulation of the system allows use to view the behavior of the pendulum for different starting conditions and how it has the possibility to stay upright.